In [24]:
import numpy as np
#import matplotlib.pyplot as plt
import re

In [ ]:
#get highest and lowest number from asc files
#global lowestNumber, highestNumber
#global lowestNumber
lowestNumber= 10000
#global highestNumber
highestNumber= -10000

def lowest_highest_asc_number(file):

    f = open(file, 'r')
    data = f.read()
    #print(data)
    pattern = r"NODATA_VALUE(.*)"
    match = re.search(pattern, data, re.DOTALL)
    global lowestNumber
    print(lowestNumber)
    global highestNumber
    print(highestNumber)
    if match:
        result = match.group(1)
        print(type(result))
        #print("Text after NODATA_VALUE:", result)

    number_array = result.split()
    print(type(number_array))

    for number in number_array:

        if (number == '-999'): continue
        int_number= int(number)
        if (int_number > highestNumber): highestNumber = int_number
        if (int_number < lowestNumber): lowestNumber = int_number

    print(lowestNumber)
    print(highestNumber)

list = ['grids_germany_annual_radiation_direct_2020.asc', 'grids_germany_annual_radiation_direct_2021.asc', 'grids_germany_annual_radiation_direct_2022.asc'
, 'grids_germany_annual_radiation_direct_2023.asc']

for file in list: 

    lowest_highest_asc_number(file)

In [3]:
import os
import requests

# get the current working directory
current_working_directory = os.getcwd()
print(current_working_directory)
os.environ['PROJ_LIB'] = '/usr/local/share/proj/proj/'

/home/nickok/Schreibtisch/SS24/CSH/grids


In [ ]:
#webscrape

import os
import requests

# get the current working directory
current_working_directory = os.getcwd()
print(current_working_directory)
os.environ['PROJ_LIB'] = '/usr/local/share/proj/proj/'

#webscrape all files: 

def download_pdf(url):

    response = requests.get(url)

    print(url)
    newUrl = url.split("/")

    newUrl = newUrl[-1].split(".")

    print(newUrl)
    name_of_file = newUrl[0]
    with open("/home/nickok/Schreibtisch/SS24/CSH/grids/" + name_of_file + ".zip", "wb") as f:
      f.write(response.content)

all_files=[]

start_year = 2015
end_year = 2025
start_month = 1

for year in range(start_year, 2024):

  download_pdf("https://opendata.dwd.de/climate_environment/CDC/grids_germany/annual/radiation_direct/grids_germany_annual_radiation_direct_" + str(year) + ".zip")
  for month in range(start_month, 13):
    if month >= 10: 
      download_pdf("https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/radiation_direct/grids_germany_monthly_radiation_direct_" + str(year) + str(month)+ ".zip")
    else:     
      download_pdf("https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/radiation_direct/grids_germany_monthly_radiation_direct_" + str(year) + "0" + str(month) + ".zip")
    
  

In [ ]:
#unpack the zipped files and delete everything up to
import os

os.environ['PROJ_LIB'] = '/usr/local/share/proj/proj/'

import zipfile

current_dir = os.getcwd()

for filename in os.listdir(current_dir):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(current_dir, filename)):

        try: 
            # Print the filename
            # Open the zip file
            filename_ending_zip = filename.split(".")
            print(filename_ending_zip)
            if(filename_ending_zip[1]=="zip"): 
                with zipfile.ZipFile(filename, 'r') as zip_ref:
                    # Extract all the contents of the zip file to the current directory
                    zip_ref.extractall()       

        except:

            print(filename)

In [ ]:
#gdal calls zum automatisierten Bearbeiten der als Subprocess, um asc in gtiff umzuwandeln

import subprocess
import re 

def gdal_translate_call(filename):

    try:
        
        filename_ending = filename.split(".")
        #print(filename_ending[0])
        #print(filename_ending[1])
        if filename_ending[1] == "asc":

            f = open(filename, 'r')
            data = f.read()
            #print(data)
            lines = data.splitlines()

            newAsciiText = ""
            after = 0
            counter= 0
            for line in lines:

                if after== 1:

                #    newAsciiText+=line
                    #print(line)
                    newAsciiText+= line+"\n"

                if "[ASCII-Raster-Format]" in line: 
                    
                    counter+=1
                    after=1

            #print(counter)
            print(newAsciiText)

            with open(filename, "w") as file:

                    file.write(newAsciiText)

            # Running a shell command and capturing its output
            result = subprocess.run(['gdal_translate -a_srs "EPSG:31467" -a_nodata -999 -ot Byte -of GTiff ' + filename + ' ' + filename_ending[0] +'.tif'], shell=True)
            print(result)
    except: 

        print("Falsche Operation")

    
def gdal_translate_call_annual(filename):

    try:
        
        filename_ending = filename.split(".")
        #print(filename_ending[0])
        #print(filename_ending[1])
        if filename_ending[1] == "asc":

            f = open(filename, 'r')
            data = f.read()
            #print(data)
            lines = data.splitlines()

            newAsciiText = ""
            after = 0
            counter= 0
            for line in lines:

                if after== 1:

                #    newAsciiText+=line
                    #print(line)
                    newAsciiText+= line+"\n"

                if "[ASCII-Raster-Format]" in line: 
                    
                    counter+=1
                    after=1

            #print(counter)
            print(newAsciiText)

            with open(filename, "w") as file:

                    file.write(newAsciiText)

            # Running a shell command and capturing its output
            result = subprocess.run(['gdal_translate -a_srs "EPSG:31467" -a_nodata -999 -ot UInt16 -of GTiff ' + filename + ' ' + filename_ending[0] +'.tif'], shell=True)
            print(result)
    except: 

        print("Falsche Operation")

current_dir = os.getcwd()

# Iterate through the files in the current directory
for filename in os.listdir(current_dir):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(current_dir, filename)):
        # Print the filename
        asc_file = filename.split(".")

        annual = asc_file[0].split("_")

        if(asc_file[-1]=="asc" and annual[2]=="annual"):

            gdal_translate_call_annual(filename)
            continue
        if(asc_file[-1]=="asc"):
        
            gdal_translate_call(filename)

In [ ]:
import math
from osgeo import gdal

colormap = {}

with open("colortable_dwd.txt", 'r') as file:
    for line in file:
        if line.startswith('INTERPOLATION') or line.startswith('#'):
            continue  # Überspringe die Überschriftenzeile
        values = line.strip().split(',')
        if values:
            value, r, g, b, alpha, extra = values
            colormap[str(value)] = [r, g, b, alpha]

# Jetzt kannst du die color_data-Liste für das nachfolgende Programm verwenden
for key in colormap:
    print(f"Wert: {colormap[key]}, Farbe: RGB({colormap[key][0]}, {colormap[key][1]}, {colormap[key][2]}), Alpha: {colormap[key][3]}")
print(colormap)

colors = gdal.ColorTable()

for key in colormap: 

    colors.SetColorEntry(math.ceil(float(key)), (int(colormap[key][0]), int(colormap[key][1]), int(colormap[key][2]), 255))  # Black

for i in range(colors.GetCount()):
    entry = colors.GetColorEntry(i)
    print(f"Color entry {i}: RGB({entry[0]}, {entry[1]}, {entry[2]})")

In [ ]:
def warp_gray(filename):

    try:    
        warpOptions = gdal.WarpOptions(format="GTiff", srcNodata=-999, dstNodata=-999, srcSRS="EPSG:31467", dstSRS="EPSG:3857")
        gdal.Warp(filename, filename, options=warpOptions)
    
    except:

        print("Gray warp failed")

current_dir = os.getcwd()

# Iterate through the files in the current directory
for filename in os.listdir(current_dir):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(current_dir, filename)):
        # Print the filename

        tif_file = filename.split(".")
        if(tif_file[-1]!="tif"):
            continue
        print(tif_file)
        tif_file=tif_file[0].split("_")

        if(tif_file[0]=="grids"):
        
            warp_gray(filename)

In [ ]:
from osgeo import gdal

def gray_to_rgb_translate_warp(filename):

    try:    
        ds = gdal.Open(filename)
        #grayCopy = driver.CreateCopy('pure_gray_'+filename, src_ds, strict=0)
        options = gdal.TranslateOptions(format="GTiff", noData=-999, rgbExpand='rgba')
        band = ds.GetRasterBand(1)
        band.SetColorTable(colors)
        print(band.GetRasterColorTable())
        ct = band.GetRasterColorTable()
        print(ct)
        print(ds.RasterCount)
        gdal.Translate('rgb_neu_'+filename, filename, options=options)
        ds_rgb = gdal.Open('rgb_neu_'+filename)
        #ds_rgb.SetGeoTransform(ds.GetGeoTransform())
        #ds_rgb.SetProjection(ds.GetProjection())

        print(ds_rgb.RasterCount)
        print(ds_rgb.GetProjection())
        
        ds_rgb = None
        ds_rgb = gdal.Open('rgb_neu_'+filename)

        print(ds_rgb.RasterCount)
        print(ds_rgb.GetProjection())
        ds = None

    except:

        print("wrong")

current_dir = os.getcwd()

# Iterate through the files in the current directory
for filename in os.listdir(current_dir):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(current_dir, filename)):
        # Print the filename

        tif_file = filename.split(".")
        if(tif_file[-1]!="tif"):
            continue
        print(tif_file)
        tif_file=tif_file[0].split("_")

        if(tif_file[0]=="grids" and tif_file[2]=="monthly"):
        
            gray_to_rgb_translate_warp(filename)

In [ ]:
####### Move all files files to project dir

#grids_germany_monthly_radiation_direct_202309.tif
#rgb_neu_grids_germany_monthly_radiation_direct_202306.tif
#/home/nickok/Schreibtisch/SS24/CSH/grids
#/home/nickok/Schreibtisch/SS24/CSH/solar_energie_map_2508/solar_energie_dw/data/geotiffs
import shutil 

for filename in os.listdir(current_dir):
    # Check if the file is a regular file (not a directory)
    if os.path.isfile(os.path.join(current_dir, filename)):
        # Print the filename

        try:
            tif_file = filename.split(".")
            if(tif_file[-1]!="tif"):
                continue
            print(tif_file)
            filename = tif_file[0]
            tif_file=tif_file[0].split("_")

            if(tif_file[0]=="grids" and tif_file[2]=="monthly"):
            
                shutil.copy2("/home/nickok/Schreibtisch/SS24/CSH/grids/"+filename+".tif", '/home/nickok/Schreibtisch/SS24/CSH/solar_energie_map_2508/solar_energie_dw/data/geotiffs/'+filename+"_3857"+".tif")
                shutil.copy2("/home/nickok/Schreibtisch/SS24/CSH/grids/"+filename+".tif", '/home/nickok/Schreibtisch/Server/flaskr/static/geotiffs/'+filename+"_3857"+".tif")  
                continue
            
            if(tif_file[0]=="rgb"):

                monthYear = filename.split("_")
                print(monthYear)
                shutil.copy2("/home/nickok/Schreibtisch/SS24/CSH/grids/"+filename+".tif", '/home/nickok/Schreibtisch/SS24/CSH/solar_energie_map_2508/solar_energie_dw/data/geotiffs/'+monthYear[-1]+"_rgb"+".tif") 
        
        except:

            print(monthYear)

In [ ]:
#Colortable einlesen und als Grafik ausgeben

In [ ]:
#Laden der Colorable

import math
from osgeo import gdal

color_table = []
index = 0

with open("color_map_jahre.txt", 'r') as file:
    for line in file:
        if line.startswith('INTERPOLATION') or line.startswith('#'):
            continue  # Überspringe die Überschriftenzeile
        values = line.strip().split(',')
        if values:
            value, r, g, b, alpha, extra = values
            color_table.append((float(value), int(r), int(g), int(b)))
            index+=1

# Jetzt kannst du die color_data-Liste für das nachfolgende Programm verwenden
for i in range(0, index):

    print(color_table[i])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# color_table = [
#     (0, 48, 18, 59),
#     (0.58594499999999994, 50, 21, 67),
#     (1.171875, 51, 24, 74),
#     # ... (Rest of your color table data) ...
#     (147.65625, 133, 7, 2),
#     (148.24214999999998099, 129, 6, 2),
#     (150, 122, 4, 3)
# ]

# Extract the kWh/m^2 values and RGB values
kwh_values = [entry[0] for entry in color_table]
rgb_values = [(entry[1]/255, entry[2]/255, entry[3]/255) for entry in color_table]

# Create a custom colormap
cmap = mcolors.ListedColormap(rgb_values)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(2, 8))

# Create a normalized color scale for the kWh values
norm = mcolors.Normalize(vmin=min(kwh_values), vmax=max(kwh_values))

# Create a color bar with the custom colormap
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax, orientation='vertical')

# Set ticks every 15 units
tick_locations = list(range(375, 870, 25))  # Ticks at every 15 units
cbar.set_ticks(tick_locations)

# Set the tick labels to the corresponding kWh values
cbar.set_ticklabels([tick for tick in tick_locations])

# Set label for the color bar
cbar.set_label('kWh/m²')

# Show the plot
plt.show()